In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization
from matplotlib import pyplot as plt # data visualization
sns.set_theme(style="darkgrid")
sns.set_palette("Set2")
plt.rcParams['figure.figsize']=(8,3)

import time
import re
import pickle

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

In [ ]:
cdf = pd.read_csv("/kaggle/input/playground-series-s4e9/train.csv")
cdf_test = pd.read_csv("/kaggle/input/playground-series-s4e9/test.csv")

In [ ]:
cdf

# **1. Knowing the Data:~🔍🏂**

In [ ]:
cdf.set_index('id', inplace=True)
cdf_test.set_index('id', inplace=True)

In [ ]:
display(cdf.columns)
display(cdf_test.columns)

In [ ]:
cdf.rename(columns={"ext_col":"ext_color", "int_col":"int_color", "milage":"mileage"}, inplace=True)
cdf_test.rename(columns={"ext_col":"ext_color", "int_col":"int_color", "milage":"mileage"}, inplace=True)

In [ ]:
display(cdf.sample(5))
display(cdf_test.sample(5))

In [ ]:
print(f"Shape of training data: {cdf.shape}, \nShape of test data: {cdf_test.shape}")

In [ ]:
cdf.isnull().sum()

In [ ]:
cdf_test.isnull().sum()

In [ ]:
display(cdf[cdf.duplicated()])
display(cdf_test[cdf_test.duplicated()])

### **SUMMARY from above //---**
* The train dataset has 13 columns, with "id" column set as index. So effectively 12. The test data has 11.
* There are **188533 rows in the train data.** While the **test dataset has 1.25lac rows.** 
* **No duplicates** in either of datset. **Both has three columns that has some null values.**
* "price" is the target column. Have **changed ext_col & int_col column names to ext_color & int_colour,** meaning Exterior & Interior Colour.

## **1.1. Details of the Numerical Columns:**

In [ ]:
cdf.describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 3), sharey=True)
fig.suptitle('Box Plots of --')

# mileage
sns.boxplot(ax=axes[0], x=cdf["mileage"])
axes[0].set_title("Mileage of Cars")

# price
sns.boxplot(ax=axes[1], x=cdf["price"])
axes[1].set_title("Price of Cars")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 4))

# mileage
sns.histplot(x=cdf["mileage"],ax=axes[0],kde=True, bins=25)
axes[0].set_title("Mileage of Cars")

# price
sns.histplot(x=cdf["price"],ax=axes[1], bins=50)
axes[1].set_title("Price of Cars")
plt.show()

In [ ]:
cdf.model_year.value_counts().head(5)

In [ ]:
plt.rcParams['figure.figsize']=(12,4)
yvc = cdf.model_year.value_counts().sort_index()
yr = sns.lineplot(x=list(yvc.index), y=yvc)
plt.xticks(list(yvc.index)[::2], rotation=90)

for i in range(0,len(yvc),2):
    yr.vlines(x=yvc.index[i],ymin=0,ymax=yvc.values[i],colors='grey',)
    yr.text(yvc.index[i],yvc.values[i], f"{yvc.values[i]}",ha='center',va="top", fontweight='bold', fontsize=9)

plt.title("No. of cars per model_year")
yr.set_yticklabels([])
plt.show()

## **1.2. Details of the Categorical Columns:**

In [ ]:
cdf.describe(include="object")

In [ ]:
cdf.accident.value_counts().plot(kind="pie", title="Reports for Accident", ylabel="",
                                y=cdf.accident.value_counts(), figsize=(3,3),
                                autopct="%1.01f%%")
plt.show()

##### Let's make a little change to replace these long values "At least 1 accident or damage reported" & "None reported"

In [ ]:
cdf.rename(columns={"accident":"accident_reported"}, inplace=True)
cdf_test.rename(columns={"accident":"accident_reported"}, inplace=True)
cdf.accident_reported = np.where(cdf.accident_reported=="None reported","No","Yes")
cdf_test.accident_reported = np.where(cdf_test.accident_reported=="None reported","No","Yes")

In [ ]:
cdf.sample(3)

In [ ]:
cdf.fuel_type.value_counts()

In [ ]:
cdf.engine.value_counts()

In [ ]:
cdf.transmission.value_counts(normalize=True)*100

In [ ]:
display(cdf.ext_color.value_counts())
display(cdf.int_color.value_counts())

In [ ]:
display(cdf.clean_title.value_counts())
cdf.clean_title.isnull().sum()

### **SUMMARY from above //---**
* **Mileage column is right skewed** as it generally should be. **Price column has a lot of outliers,** probably fo the presence of a few supar cars.
* There are 12 cars of model year 1974. Most cars are of years between 2015 & 2022.
* **23% cars have at least one accident or damage being reported.**
* **`engine` & `transmission` columns have so many unique values.** Some column transformation can be done on those columns like creating a few smaller columns like Cylinder No, or Capacity etc.
* Color columns have many unique values as well, black being the most frequent for both Exterior and Interior color.

# **2. Data Cleaning and Processing:~ 🧹🏗️**

In [ ]:
cdf.isnull().sum()

## **2.1. Managing the `Fuel type` column** --

In [ ]:
cdf.fuel_type.unique()

In [ ]:
cdf_test.fuel_type.unique()

### >> Data transformation ~

In [ ]:
cdf['fuel_type'] = cdf['fuel_type'].replace(['–', 'not supported'], 'None')
cdf_test['fuel_type'] = cdf_test['fuel_type'].replace(['–', 'not supported'], 'None')

### >> Null Value Handling ~ 

In [ ]:
cdf.fuel_type = cdf.fuel_type.fillna("Unknown")
cdf_test.fuel_type = cdf_test.fuel_type.fillna("Unknown")

In [ ]:
cdf.fuel_type = cdf.fuel_type.apply(lambda x:x.replace(" ","_")).apply(lambda x:x.replace("-","_"))
cdf_test.fuel_type = cdf_test.fuel_type.apply(lambda x:x.replace(" ","_")).apply(lambda x:x.replace("-","_"))

## **2.2. Managing the `Transmission` column** --

In [ ]:
cdf.transmission.unique()

### >> Data Extraction ~
#### Transmission column have **more than 50 unique values** in it. That is because 2-3 different data are clubbed together in this column. So, those information like **no. of gears & transmission type are extracted from it.**

In [ ]:
cdf.transmission = cdf.transmission.replace({"Single-Speed Fixed Gear":"1-Speed Fixed Gear"})
cdf_test.transmission = cdf_test.transmission.replace({"Single-Speed Fixed Gear":"1-Speed Fixed Gear"})

In [ ]:
def extract_gear_and_txtype(transmission_info):
    pattern = re.search(r'(\d{1,2}[\s-]?speed?)?\s*(Automatic|Electronically Controlled Automatic|At\/Mt|A\/T|AT|M\/T|CVT|Manual|Variable|Transmission Overdrive|Fixed|DCT|Mt|Transmission w/Dual Shift Mode)?\s*',transmission_info,re.IGNORECASE)
    
    gear = pattern.group(1) if pattern.group(1) else None
    txtype = pattern.group(2) if pattern.group(2) else "Other"
    return gear, txtype

def load_gear_and_txtype(df):
    gear = []
    transmission_type = []
    for tx in df.transmission:
        ngear,txtype = extract_gear_and_txtype(tx)

        if ngear!=None:
            ngear = ngear.split("-")[0].split(" ")[0] #to tackle both 6-speed & 6 speed
            if ngear.lower()=="single":ngear=1
            else:ngear = int(ngear)

        if txtype!=None:
            if txtype=="At/Mt": txtype="AMT"
            elif txtype.lower() in ['a/t','at','transmission overdrive']: txtype = "Automatic"
            elif txtype.lower() in ['m/t','mt']: txtype = "Manual"
            elif txtype.lower()=="variable": txtype="CVT"
            elif txtype=="Transmission w/Dual Shift Mode": txtype="Dual_Shift"
            elif txtype=="Electronically Controlled Automatic": txtype="Electronically_controlled"

        gear.append(ngear)
        transmission_type.append(txtype)


    df["gears"] = gear
    df["transmission_type"] = transmission_type
    
    return df


In [ ]:
cdf = load_gear_and_txtype(cdf)
cdf_test = load_gear_and_txtype(cdf_test)

In [ ]:
display(cdf.gears.unique())
display(cdf_test.gears.unique())

In [ ]:
display(cdf.transmission_type.unique())
display(cdf_test.transmission_type.unique())

In [ ]:
cdf.transmission_type.value_counts()

## **2.3. Working with the `Engine` column** --

In [ ]:
cdf.engine.nunique()

In [ ]:
cdf.engine.sample(5)

### >> Data Extraction ~
#### Engine column too have a lot of unique values in it **(more than 50 unique values).** Again, this is because 3-4 different information are concatenated in this column. So, data related to **HorsePower, Capacity, & no. of Cylinders are extracted from it.**

In [ ]:
def extract_engine_data(engine_info):
    pattern = re.search(r'(\d{1,4}.\d{1,2}HP)?\s?(\d{1,2}.\d{1,2}[L|\sLiter])?[A-Za-z\s]{0,}(\d{1,2})?\s?[Cylinder]{0,}',engine_info,re.IGNORECASE)
    
    hp = pattern.group(1) if pattern.group(1) else None
    capacity = pattern.group(2) if pattern.group(2) else None
    cylinders = pattern.group(3) if pattern.group(3) else None
    return hp, capacity, cylinders

def load_engine_data(df):
    horsepower, capacity, cylinders = [],[],[]
    i=0
    for engine in df.engine:
        hp, cap, cy = extract_engine_data(engine)
        if hp!=None:
            hp = float(hp.lower().split("hp")[0]) 

        if cap!=None:
            cap = float(cap.lower().split("l")[0])    

        if cy!=None:
            cy = float(cy.split(" ")[0])

        horsepower.append(hp)
        capacity.append(cap)
        cylinders.append(cy)


    df["horsepower"] = horsepower
    df["engine_capacity"] = capacity
    df["cylinders"] = cylinders
    
    return df

In [ ]:
cdf = load_engine_data(cdf)
cdf_test = load_engine_data(cdf_test)

In [ ]:
cdf[['engine','horsepower','engine_capacity','cylinders']].sample(5)

In [ ]:
cdf_test[['engine','horsepower','engine_capacity','cylinders']].sample(5)

##### nan values in the newly added columns will be fixed later.

## **2.4. Fixing the `Exterior & Interior Color` columns** --

In [ ]:
cdf.ext_color.nunique()

In [ ]:
cdf.ext_color.value_counts(normalize=True).head(15)

In [ ]:
cdf.int_color.value_counts(normalize=True).head(15)

### >> Data transformation ~
#### Colors columns are having many unique values as well because different color shades are present of a same parent color, e.g. both "Jet Black" & "Black" is present in the data. 
##### 
#### **The plan is to keep some of the common base colors. If a color has that base color in it, it will be changed to the base color, or else the color will be made uncommon,** i.e. "Jet Black" will be transformed to "Black" itself, but suppose we are not considering "pink" as our base color, so it will become 'uncommon'. 

In [ ]:
base_colors = ['white','black','grey','gray','blue','red','yellow','silver','green','beige','gold','orange','brown','ebony','purple']

def find_base_color(text):
        for color in base_colors:
            if color in text:
                return color
        return "uncommon"  
    
def transform_color(df):    
    df.ext_color = df.ext_color.apply(lambda x: x.lower() if isinstance(x, str) else x)
    df.int_color = df.int_color.apply(lambda x: x.lower() if isinstance(x, str) else x)
    df.ext_color = df.ext_color.apply(find_base_color)
    df.int_color = df.int_color.apply(find_base_color)
    df.ext_color = df.ext_color.replace({"grey":"gray"})
    df.int_color = df.int_color.replace({"grey":"gray"})
    
    return df

cdf = transform_color(cdf)
cdf_test = transform_color(cdf_test)

In [ ]:
cdf.ext_color.unique()

In [ ]:
cdf.ext_color.value_counts(normalize=True)*100

In [ ]:
cdf.int_color.value_counts(normalize=True)*100

In [ ]:
cdf.sample(2)

## **2.5. Dealing with the `clean title` columns** --

In [ ]:
display(cdf.clean_title.value_counts())
print("----")
display(cdf_test.clean_title.value_counts())

In [ ]:
cdf[['accident_reported','clean_title']].value_counts()

### >> Handling Null values ~
#### Logically clean title is dependent on accident/damage reports. If there are any Severe damage, its value is false. But cars with no reports of accident or minimal accidents... generally has a clean title. The above table show this.
#####  
#### So, to handle the Null values in this column, will take reference from the accident_reported column.

In [ ]:
def fill_clean_title(row):
    if pd.isna(row['clean_title']):
        if row['accident_reported'] == 'No':
            return "Yes"
        elif row['accident_reported'] == 'Yes':
            return "No"
    return row['clean_title']

# Apply the function to each row
cdf.clean_title = cdf.apply(fill_clean_title, axis=1)
cdf_test.clean_title = cdf_test.apply(fill_clean_title, axis=1)

In [ ]:
cdf[['accident_reported','clean_title']].value_counts()

In [ ]:
cdf_test[['accident_reported','clean_title']].value_counts()

## **2.6. Managing the `brand & Model` columns** --

In [ ]:
cdf.brand.unique()

In [ ]:
cdf.model.nunique()

In [ ]:
cdf[['brand','model']].value_counts().head(50)

In [ ]:
cdf[["brand","price"]].groupby('brand').median(['price']).sort_values("price",ascending=False).head(10)

### >> Data transformation ~
#### Have classified the cars into 5 classes based on the brand's price, performance and utilizations. This is feature reduction as well as to maintain the brand-value as is.

In [ ]:
# Car brand categories
categories = {
    'Exotic_car': [
        'Bugatti', 'Ferrari', 'Lamborghini', 'McLaren', 'Rolls-Royce',
        'Bentley', 'Aston', 'Koenigsegg', 'Pagani'
    ],
    'Super_car': ['Porsche', 'Maserati', 'Lotus'],
    'Luxury_car': [
        'Mercedes-Benz', 'Audi', 'BMW', 'Genesis', 'Cadillac', 'Lincoln',
        'Land', 'Jaguar', 'Tesla', 'Lexus', 'INFINITI', 'Acura',
        'Polestar', 'Maybach'
    ],
    'Premium_car': [
        'Volvo', 'Volkswagen', 'Buick', 'Rivian', 'RAM', 'Alfa', 'Jeep'
    ],
    'Standard_car': [
        'MINI', 'Chevrolet', 'Ford', 'GMC', 'Toyota', 'Hyundai', 'Kia', 
        'Mitsubishi', 'Honda', 'Nissan', 'Mazda', 'Subaru', 'Chrysler', 'Lucid', 
        'Scion', 'smart', 'Karma', 'Plymouth', 'Suzuki', 'FIAT', 'Saab', 
        'Pontiac', 'Saturn', 'Dodge', 'Hummer', 'Mercury'
    ]
}

# Function to classify brands
def classify_brand(brand):
    for category, brands in categories.items():
        if brand in brands:
            return category
    return "Other"

# Apply classification to the DataFrame
cdf['car_category'] = cdf.brand.apply(classify_brand)
cdf_test['car_category'] = cdf_test.brand.apply(classify_brand)


In [ ]:
cdf_test.car_category.value_counts()

In [ ]:
cdf.sample(5)

## **2.7. Handling Null values in NEW Numerical columns** --

#### The plan is to fill the null values with the mode of corresponding columns of those rows that have the same brand and model name. If that also is null, null is filled with mode of same brand name only. 

In [ ]:
cdf.isnull().sum()

In [ ]:
# def fill_gears_na(df, row):
#     same_brand = row['brand']
#     same_model = row['model']

#     try:
#         same_gears = df[(df.brand==same_brand) & (df.model==same_model)].gears.mode()[0]
#         return same_gears
#     except:
#         return df[df.brand==same_brand].gears.mode()[0]       


# for i in range(len(cdf)):
#     if pd.isna(cdf.loc[i,'gears']):
#         cdf.loc[i,'gears'] = fill_gears_na(cdf, cdf.loc[i,:])


## THIS TAKES LOT OF TIME TO RUN

In [ ]:
cdf

In [ ]:
def fill_na_with_same_brand_model_mode(df, grouping_cols, target_cols):
    
    # Mode Calculator
    def calculate_mode(series):
        mode = series.mode()
        return mode.iloc[0] if not mode.empty else np.nan
    
    # NA filler function
    def impute_na(row):
        if pd.notna(row[col]):
            return row[col]
        if pd.notna(row['brand_model_mode']):
            return row['brand_model_mode']
        if pd.notna(row['brand_mode']):
            return row['brand_mode']

        return row[col]
    
    #-------------
    for col in target_cols:
        # find mode of same brand & model
        brand_model_mode = df.groupby(grouping_cols)[col].apply(calculate_mode).rename("brand_model_mode")
        
        # find mode of same brand
        brand_mode = df.groupby(grouping_cols[0])[col].apply(calculate_mode).rename("brand_mode")
        
        #merging the series to have corresponding mode for each brand & band_model combo in each rows
        df = df.merge(brand_model_mode, on=grouping_cols, how='left')
        df = df.merge(brand_mode, on=grouping_cols[0], how='left')
        
        #applying na_filler function
        df[col] = df.apply(impute_na, axis=1)
        
        #deleting newly added cols
        df.drop(columns=['brand_model_mode','brand_mode'],inplace=True)
    
    return df
 

In [ ]:
cdf = fill_na_with_same_brand_model_mode(cdf, ['brand','model'], ['gears','horsepower','engine_capacity','cylinders'])
cdf_test = fill_na_with_same_brand_model_mode(cdf_test, ['brand','model'], ['gears','horsepower','engine_capacity','cylinders'])

In [ ]:
cdf.isnull().sum()

In [ ]:
# 10 rows had null in them. All for same model: Bugatti--Veyron 16.4 Grand Sport: 1200 HP checked from google and filled.

cdf.horsepower = cdf.horsepower.fillna(1200)
cdf_test.horsepower = cdf_test.horsepower.fillna(1200)

## **2.8. New Column in place of `Model year`** --

#### year as in 2002, 2020, 2023 doesn't have any numerical info in it. It is more like a categorical column only. So, a new column `car_age` is created to have that longivity feel for the cars.

In [ ]:
cdf['car_age'] = 2024-cdf.model_year
cdf_test['car_age'] = 2024-cdf_test.model_year

In [ ]:
cdf.sample(5)

## **2.9. Dropping unwanted columns** --

#### As a result of the above part, EDA-2 is done. We have handled all Null values, processed & transformed columns to modify the existing columns... as well as to create a few new ones. 
* Let's remove the redundant columns now. &
* modify the dataset view.

In [ ]:
cdf.drop(columns=['brand','model','model_year','engine','transmission'], inplace=True)
cdf_test.drop(columns=['brand','model','model_year','engine','transmission'], inplace=True)

In [ ]:
cdf = cdf[['car_category', 'fuel_type', 'transmission_type', 'int_color', 'ext_color', #cat_cols
           'accident_reported', 'clean_title', #binary_cols
           'car_age', 'mileage', 'gears', 'horsepower', 'engine_capacity', 'cylinders', 'price']] #num_cols

cdf_test = cdf_test[['car_category', 'fuel_type', 'transmission_type', 'int_color', 'ext_color', #cat_cols
                     'accident_reported', 'clean_title', #binary_cols
                     'car_age', 'mileage', 'gears', 'horsepower', 'engine_capacity', 'cylinders']] #num_cols

In [ ]:
cdf_test

# **3. Understanding inter-column relationships:~ 📈🐾**

In [ ]:
plt.rcParams['figure.figsize']=(8,3)

## **3.1. Univariate Analysis $^~|%**

### **3.1.1: car category**

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12,4))

cdf.car_category.value_counts().plot(ax=axes[0], kind="pie", y=cdf.car_category.value_counts(), autopct = "%1.01f%%", 
                             title="Proportion of Car types in `cdf` Data\n", ylabel="", startangle=30)

cdf_test.car_category.value_counts().plot(ax=axes[1], kind="pie", y=cdf_test.car_category.value_counts(), startangle=30,
                              autopct = "%1.01f%%", title="Proportion of Car types in `cdf_test` Data\n", ylabel="")
plt.show()

### **3.1.2: fuel type**

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10,4.5), sharex=True)

fig.suptitle("Percentage occurence of different fuel types")
p1 = (cdf.fuel_type.value_counts(normalize=True)*100).plot(ax=axes[0], kind="barh", 
                                                           y=cdf.fuel_type.value_counts(), xlabel="", ylabel="")

p2 = (cdf_test.fuel_type.value_counts(normalize=True)*100).plot(ax=axes[1],kind="barh",
                                                                y=cdf_test.fuel_type.value_counts(), xlabel="",ylabel="")

p1.text(28,3.2,"in `cdf` data",color='gray')
p2.text(28,3.2,"in `cdf_test` data", color="grey")
plt.xticks([2,5,25,50,75,90,100])
plt.show()

### **3.1.3: transmission type**

In [ ]:
cdf.transmission_type.value_counts(normalize=True)*100

In [ ]:
freq_txtype = ['Automatic','Dual_Shift','Manual','CVT']
fig, axes = plt.subplots(1, 2, figsize=(12,4), sharex=True)

cdf[cdf.transmission_type.isin(freq_txtype)].transmission_type.value_counts().plot(ax=axes[0], 
                            kind="pie", radius=1, autopct = "%1.01f%%", startangle=30,
                            y=cdf.transmission_type.value_counts(), title="Transmission types in `cdf`", ylabel="")

cdf_test[cdf_test.transmission_type.isin(freq_txtype)].transmission_type.value_counts().plot(ax=axes[1], kind="pie",
                            radius=1, autopct = "%1.01f%%", y=cdf_test.transmission_type.value_counts(), startangle=220,
                            title="Transmission types in `cdf_test`", ylabel="")

plt.show()

### **3.1.4: all numerical columns**

In [ ]:
import random
colors=['#286aad','#296c6c','#533b84','#9f1455','#5b7231']

num_cols = ['car_age', 'mileage','gears','horsepower','engine_capacity','cylinders']
fig, axes = plt.subplots(2, 3, figsize=(15, 6), sharey=True)
fig.suptitle('Violin plots of --', fontsize=17)

for i,row in enumerate(np.reshape(num_cols,(2,3))):
    for j,col in enumerate(row):
        a=sns.violinplot(ax=axes[i][j], x=cdf[row[j]],color=random.choices(colors,k=1)[0])
        a.text(cdf[row[j]].max()/3,.4,
               f"skew={round(cdf[row[j]].skew(),2)}, median={round(cdf[row[j]].median(),2)}",
               color=random.choices(colors,k=1)[0])
    
fig.tight_layout()

In [ ]:
cdf['engine_capacity'].unique()

In [ ]:
cdf['engine_capacity'].mask(cdf['engine_capacity'] == 120, cdf['engine_capacity'].median(), inplace=True)
sns.scatterplot(x=cdf[col], y=cdf.price,color='#0a7e84') 
plt.show()

cdf_test['engine_capacity'].mask(cdf_test['engine_capacity'] == 120, cdf_test['engine_capacity'].median(), inplace=True)

## **3.2. Bi-variate Analysis $^~|%**

In [ ]:
cdf.sample(1)

### **3.2.1: price with car's category**

In [ ]:
sns.boxplot(x=cdf.price,y=cdf.car_category)
plt.show()

### **3.2.2: price with transmission type**

In [ ]:
sns.boxplot(x=cdf.price,y=cdf.transmission_type)
plt.show()

### **3.2.3: price with numerical columns**

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 7), sharey=True)
fig.suptitle('Scatter plots between Price & --', fontsize=17)

for i,row in enumerate(np.reshape(num_cols,(2,3))):
    for j,col in enumerate(row):
        a=sns.scatterplot(ax=axes[i][j], x=cdf[col], y=cdf.price,color=random.choices(colors,k=1)[0])
        a.set_xlabel(col, fontsize=15)
        axes[i][j].tick_params(axis='x',labelrotation=25) # axes[i][j].set_xticklabels(labels, rotation=45) works as well, 
                                                 # but labels were required. That's possible when we have 
                                                 # a specified list of labels. Here the labels are selected automatically
                                                 # default value of x is 'both', rotates both x & y
    
fig.tight_layout()   

### **3.2.4: top_20 most frequent color combinations**

In [ ]:
top_color_combo = (cdf[['ext_color','int_color']].value_counts(normalize=True)*100).head(20).reset_index().sample(20)
top_color_combo['ext_int_colors'] = top_color_combo.ext_color+"_"+top_color_combo.int_color
color_plt = sns.barplot(x=top_color_combo.ext_int_colors, y=top_color_combo.proportion,width=0.6)
plt.xticks(rotation=90)
plt.show()

### **3.2.5: car category with all numerical columns**

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
fig.suptitle('Box plots between Car category & --', fontsize=17)

for i,row in enumerate(np.reshape(num_cols,(2,3))):
    for j,col in enumerate(row):
        a=sns.boxplot(ax=axes[i][j], x=cdf[row[j]],y=cdf.car_category, color=random.choices(colors,k=1)[0])
    
fig.tight_layout()  
plt.ylabels = []
plt.show()

### **3.2.6: most common fuel & transmission type combo**

In [ ]:
top_tf_combo = (cdf[['fuel_type','transmission_type']].value_counts(normalize=True)*100).head(8).reset_index()
top_tf_combo['fuel_tx_combo'] = top_tf_combo.fuel_type+"_"+top_tf_combo.transmission_type

plt.pie(top_tf_combo.proportion,labels=top_tf_combo.fuel_tx_combo,startangle=-10)
plt.title("Top-8 most common fuel & transmission type combo",color='indigo')
plt.show()

### **3.2.7: distribution of fuel types & transmission types for each categories of car**

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(18,4))
car_category_fuel_contingency1 = pd.crosstab(cdf.car_category, cdf.fuel_type)
a = car_category_fuel_contingency1.plot(ax=axes[0], kind='bar', stacked=True,width=0.5)
a.text(3,80000,"in `cdf` data",fontsize=13,color='grey',fontweight='bold')

car_category_fuel_contingency2 = pd.crosstab(cdf_test.car_category, cdf_test.fuel_type)
b = car_category_fuel_contingency2.plot(ax=axes[1],kind='bar', stacked=True,width=0.5)
b.text(3,53000,"in `cdf_test` data",fontsize=13,color='grey',fontweight='bold')

axes[0].legend(title='Types of Fuel\n', loc='upper center')
axes[1].legend(title='Types of Fuel\n', loc='upper center')
plt.show()

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(18,4))
car_category_tx_contingency1 = pd.crosstab(cdf.car_category, cdf.transmission_type).sort_values('Automatic')
a = car_category_tx_contingency1.plot(ax=axes[0], kind='bar', stacked=True,width=0.8)
a.text(1.8,80000,"in `cdf` data",fontsize=13,color='grey',fontweight='bold')

car_category_tx_contingency2 = pd.crosstab(cdf_test.car_category, cdf_test.transmission_type).sort_values('Automatic')
b = car_category_tx_contingency2.plot(ax=axes[1],kind='bar', stacked=True,width=0.8)
b.text(1.8,53000,"in `cdf_test` data",fontsize=13,color='grey',fontweight='bold')

axes[0].legend(title='Transmission types', loc='upper left')
axes[1].legend(title='Transmission types', loc='upper left')
plt.show()

### **3.2.8: distribution of accident_reported & clean_title columns for each car type**

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(12,3),sharey=True)

sns.countplot(ax=axes[0],data=cdf, x='accident_reported', hue='car_category')
sns.countplot(ax=axes[1],data=cdf, x='clean_title', hue='car_category')
plt.show()

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(12,2))
accident_cleantitle_contingency1 = pd.crosstab(cdf.clean_title, cdf.accident_reported)
a = accident_cleantitle_contingency1.plot(ax=axes[0], kind='bar', stacked=True,width=0.3)
a.text(0.2,45000,"in `cdf` data",fontsize=10,backgroundcolor='white')
accident_cleantitle_contingency2 = pd.crosstab(cdf_test.clean_title, cdf_test.accident_reported)
b = accident_cleantitle_contingency2.plot(ax=axes[1],kind='bar', stacked=True,width=0.3)
b.text(0.2,35000,"in `cdf_test` data",fontsize=10,backgroundcolor='white')
plt.suptitle("accident_reported 'stacked'... for each clean_title bars",color="indigo",fontsize=12)
plt.show()

### **3.2.9: a few Numerical columns among themselves**

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(15,7))

sns.scatterplot(ax=axes[0,0],data=cdf,x='horsepower',y='mileage',color='#921A40')
sns.scatterplot(ax=axes[0,1],data=cdf,x='horsepower',y='engine_capacity',color='#0B8494')
sns.scatterplot(ax=axes[1,0],data=cdf,x='horsepower',y='cylinders',color='#5A639C')
sns.scatterplot(ax=axes[1,1],data=cdf,x='car_age',y='mileage',color='#CB80AB')
fig.tight_layout()

### **3.2.10: fuel & transmission type's influence on numerical column**

In [ ]:
sns.boxplot(data=cdf,x='fuel_type',y='engine_capacity')
plt.show()

In [ ]:
sns.boxplot(data=cdf,x='horsepower',y='transmission_type')
plt.show()

In [ ]:
sns.boxplot(data=cdf,x='fuel_type',y='mileage')
plt.show()

### **3.2.11: Average Price and mileage trend per various types**

In [ ]:
# Average Price & Mileage for categories
avg_per_category = cdf[['car_category','price','mileage']].groupby('car_category').mean().sample(5)
avg_per_category.plot(kind='line',linewidth=2.2,linestyle='-.',marker='o')
plt.show()

In [ ]:
avg_per_fuel_type = cdf[['fuel_type','price','mileage']].groupby('fuel_type').mean().sample(7)
avg_per_fuel_type.plot(kind='line',linewidth=2.2,linestyle='-.',marker='o',figsize=(12,3))
plt.show()

In [ ]:
avg_per_transmission_type = cdf[['transmission_type','price','mileage']].groupby('transmission_type').mean().sample(9)
avg_per_transmission_type.plot(kind='line',linewidth=3,linestyle='-.',marker='o',figsize=(18,4))
plt.show()

In [ ]:
hp_per_category = cdf[['car_category','horsepower']].groupby('car_category').mean()
hp_per_tx = cdf[['transmission_type','horsepower']].groupby('transmission_type').mean()
hp_per_fueltype = cdf[['fuel_type','horsepower']].groupby('fuel_type').mean()
hp_trend = pd.concat([hp_per_category,hp_per_tx,hp_per_fueltype])

plt.figure(figsize=(15,1))
sns.heatmap(hp_trend.T,cmap="coolwarm",linewidths=0.8,linecolor='white')
plt.show()

## **3.3. Multi-variate Analysis $^~|%**

In [ ]:
cdf.sample(2)

In [ ]:
sns.scatterplot(data=cdf,x='price',y='horsepower',hue='engine_capacity', palette='viridis')
plt.show()

In [ ]:
sns.scatterplot(data=cdf,x='horsepower',y='car_age',hue='gears', palette='summer')
plt.show()

In [ ]:
sns.scatterplot(data=cdf,x='mileage',y='car_age',hue='accident_reported')
plt.show()

In [ ]:
sns.scatterplot(data=cdf,x='horsepower',y='engine_capacity',hue='fuel_type')
plt.show()